# Gene trees

## Preparation

In [10]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

In [11]:
project_path=Path().resolve().parent
genomes_path=project_path / "results" / "intermediate" / "filtered_ffns"
orthogroup_seq_path=project_path / "results" / "intermediate" / "gene_tree_sequences"

In [12]:
pangenome=pd.read_csv(project_path / "results" / "intermediate" / "filtered_pangenome.csv")

In [13]:
pangenome

,Unnamed: 0,genome,gtdb_species,counts,gene,orthogroup
0,0,GCA_000687335.1,Apilactobacillus kunkeei_A,10,AZBY01000002.1_55,F00268_1
1,1,GCA_000687335.1,Apilactobacillus kunkeei_A,10,AZBY01000006.1_5,F00239_44
2,2,GCA_000687335.1,Apilactobacillus kunkeei_A,10,AZBY01000017.1_5,F00239_42
3,3,GCA_000687335.1,Apilactobacillus kunkeei_A,10,AZBY01000022.1_5,F00239_42
4,4,GCA_000687335.1,Apilactobacillus kunkeei_A,10,AZBY01000005.1_59,F00476_30
...,...,...,...,...,...,...
7173240,7173240,GCA_900166935.1,Weissella confusa,21,FUWE01000032.1_2,F20199_1
7173241,7173241,GCA_900166935.1,Weissella confusa,21,FUWE01000032.1_3,F09484_2
7173242,7173242,GCA_900166935.1,Weissella confusa,21,FUWE01000032.1_4,F24229_1
7173243,7173243,GCA_900166935.1,Weissella confusa,21,FUWE01000032.1_5,F21546_1


## collect gene sequences per orthogroup

loop over genomes (in pangenome)

loop over genes (in genome.ffn)

write sequence file

In [14]:
orthogroup="F00691_06"
gene="AEBA01000134.1_40"
genome="GCA_000179475.1"

In [15]:
genome_file=genome+".ffn"

In [16]:
def read_ffn(genome):
    genome_file=genome+".ffn"
    df = pd.read_table(
    genomes_path / genome_file,
    engine = 'c',
    lineterminator = '>',
    skiprows =1,
    names = ['raw']
    )

    # The first line break ('\n') separates Column 0 from Column 1
    df[['header','sequence']] = pd.DataFrame.from_records(df.raw.apply(lambda s: s.split(maxsplit=1, sep="\n")))

    # All subsequent line breaks (which got left in Column 1) should be ignored
    df['sequence'] = df['sequence'].apply(lambda s: s.replace('\n',''))
    
    # Only the first word of the header is relevant.
    df['gene'] = pd.DataFrame.from_records(df.header.apply(lambda s: s.split(maxsplit=1)))[0]
    return df[['gene', 'sequence']]

In [16]:
def get_sequence(df, gene):
    sequence=df.sequence[df.gene==gene].item()
    return sequence

In [17]:
def add_to_file(orthogroup, gene, genome, sequence, species):
    file_title = orthogroup_seq_path / species / str(orthogroup+".txt")
    file_object = open(file_title, 'a')
    file_object.write(str(">"+genome+"-"+gene+"\n"))
    file_object.write(str(sequence+"\n"))
    file_object.close()

In [52]:
# Very slow

def add_files(genome, species):
    df=read_ffn(genome)
    length=len(df)
    j=0
    for gene in df.gene:
        j=j+1
        orthogroup=pangenome.orthogroup[pangenome.gene==gene].item()
        #print('orthogroup found')
        add_to_file(orthogroup, gene, genome, df.sequence[df.gene==gene].item(), species)
        #print('orthogroup added to file')
        print("done: " + str(j) + "/" + str(length))

In [18]:
def add_files(genome, species):
    df=read_ffn(genome)
    length=len(df)
    for j in range(0, df.shape[0]):
        add_to_file(pangenome.orthogroup.iloc[j], pangenome.gene.iloc[j], genome, df.sequence.iloc[j], species)

In [19]:
i=0

In [21]:
for species in pangenome.gtdb_species.drop_duplicates():
    for genome in pangenome.genome[pangenome.gtdb_species == species].drop_duplicates():
        add_files(genome, species)
        print(str(genome + " done!"))
    print(str(species + " done"))

GCA_000687335.1 done!
GCA_006493415.1 done!
GCA_001421135.1 done!
GCA_001421115.1 done!
GCA_001281285.1 done!
GCA_001949975.1 done!
GCA_001281215.1 done!
GCA_001281205.1 done!
GCA_000830375.1 done!
GCA_001281225.1 done!
Apilactobacillus kunkeei_A done
GCA_001715775.1 done!
GCA_900332075.1 done!
GCA_900332065.1 done!
GCA_900332055.1 done!
GCA_900218095.1 done!
GCA_002442515.1 done!
GCA_001716015.1 done!
GCA_001715735.1 done!
GCA_001715665.1 done!
GCA_001715645.1 done!
GCA_001703535.1 done!
GCA_001703515.1 done!
GCA_000620985.1 done!
GCA_000525935.1 done!
GCA_003537455.1 done!
GCA_001715725.1 done!
Brochothrix thermosphacta done
GCA_003264045.1 done!
GCA_901830375.1 done!
GCA_003264145.1 done!
GCA_003264135.1 done!
GCA_003264085.1 done!
GCA_003264065.1 done!
GCA_003264015.1 done!
GCA_003263975.1 done!
GCA_003263965.1 done!
GCA_003263915.1 done!
GCA_002005795.1 done!
GCA_000245815.1 done!
GCA_007197715.1 done!
GCA_003264005.1 done!
Dolosigranulum pigrum done
GCA_004125955.1 done!
GCA_0039

GCA_001025385.1 done!
GCA_001808975.1 done!
GCA_001812615.1 done!
GCA_001814285.1 done!
GCA_001696305.1 done!
Enterococcus_B faecium_B done
GCA_002265245.1 done!
GCA_003321045.1 done!
GCA_002278015.2 done!
GCA_002360145.1 done!
GCA_002930795.1 done!
GCA_003321025.1 done!
GCA_003796045.1 done!
GCA_004793905.1 done!
GCA_004099875.1 done!
GCA_004099965.1 done!
GCA_004167465.1 done!
GCA_005047985.1 done!
GCA_900636615.1 done!
GCA_002077575.1 done!
GCA_004099685.1 done!
GCA_002077545.1 done!
GCA_000391765.1 done!
GCA_002077355.1 done!
GCA_002212045.1 done!
GCA_002213345.1 done!
GCA_002213365.1 done!
GCA_002213385.1 done!
GCA_002403695.1 done!
GCA_002077385.1 done!
GCA_901543435.1 done!
GCA_901543075.1 done!
GCA_000407425.1 done!
GCA_000699565.1 done!
GCA_000767355.1 done!
GCA_001280825.1 done!
GCA_001641305.1 done!
GCA_002077295.1 done!
GCA_002077325.1 done!
Enterococcus_B hirae done
GCA_003048275.1 done!
GCA_002944895.1 done!
GCA_002216705.1 done!
GCA_005158285.1 done!
GCA_001705215.1 done

GCA_004683785.1 done!
GCA_004102845.1 done!
GCA_004101605.1 done!
GCA_002943545.1 done!
GCA_004101505.1 done!
GCA_004078645.1 done!
GCA_004028315.1 done!
GCA_003999275.1 done!
GCA_003813125.1 done!
GCA_003709415.1 done!
GCA_003597635.1 done!
GCA_003346235.1 done!
GCA_003346075.1 done!
GCA_003345375.1 done!
GCA_003097595.1 done!
GCA_004087995.1 done!
GCA_001908455.1 done!
GCA_002117265.1 done!
GCA_001888505.1 done!
GCA_001273585.1 done!
GCA_001272315.2 done!
GCA_000966475.1 done!
GCA_000931425.2 done!
GCA_000830535.1 done!
GCA_000731855.1 done!
GCA_000687495.1 done!
GCA_000648755.1 done!
GCA_000507045.1 done!
GCA_000473935.1 done!
GCA_001302645.1 done!
GCA_000410795.1 done!
GCA_000203855.3 done!
GCA_900618215.1 done!
GCA_900095045.1 done!
GCA_003045725.1 done!
GCA_003045665.1 done!
GCA_002906095.1 done!
GCA_001435215.1 done!
GCA_000615325.1 done!
GCA_000604145.1 done!
GCA_000466845.1 done!
GCA_000247735.2 done!
GCA_001368775.1 done!
GCA_001888525.1 done!
GCA_001595615.1 done!
GCA_001888

GCA_000731635.1 done!
GCA_000805435.1 done!
GCA_000236475.1 done!
Lactococcus lactis_E done
GCA_900178545.1 done!
GCA_005049195.1 done!
GCA_004088235.1 done!
GCA_003957415.1 done!
GCA_003410375.1 done!
GCA_003254785.1 done!
GCA_002224505.1 done!
GCA_002224425.1 done!
GCA_001981925.1 done!
GCA_001982025.1 done!
GCA_001723545.1 done!
GCA_001698165.1 done!
GCA_001663835.1 done!
GCA_001435495.1 done!
GCA_000805355.1 done!
GCA_000235705.2 done!
GCA_003966815.1 done!
GCA_001311645.1 done!
GCA_001981905.1 done!
Latilactobacillus curvatus done
GCA_004099995.1 done!
GCA_006538665.1 done!
GCA_900215765.1 done!
GCA_900215775.1 done!
GCA_900215905.1 done!
GCA_900216205.1 done!
GCA_900234405.1 done!
GCA_900234355.1 done!
GCA_900234375.1 done!
GCA_900234395.1 done!
GCA_900290175.1 done!
GCA_900408275.1 done!
GCA_003627235.1 done!
GCA_900234345.1 done!
GCA_901830485.1 done!
GCA_003288235.1 done!
GCA_003288195.1 done!
GCA_003288215.1 done!
GCA_000026065.1 done!
GCA_000478625.2 done!
GCA_000755625.1 do

GCA_002128635.1 done!
GCA_002128515.1 done!
GCA_002128495.1 done!
GCA_002128485.1 done!
GCA_002128525.1 done!
GCA_002112245.1 done!
GCA_002112225.1 done!
GCA_002112185.1 done!
GCA_002027295.1 done!
GCA_002007085.1 done!
GCA_002112805.1 done!
GCA_001889975.1 done!
Limosilactobacillus reuteri done
GCA_002527095.1 done!
GCA_002528035.1 done!
GCA_002528595.1 done!
GCA_002557645.1 done!
GCA_002915005.2 done!
GCA_002966935.1 done!
GCA_003186965.1 done!
GCA_003186985.1 done!
GCA_002749155.1 done!
GCA_002526155.1 done!
GCA_001827615.1 done!
GCA_002524065.1 done!
GCA_002445265.1 done!
GCA_002105715.1 done!
GCA_001999085.1 done!
GCA_001827625.1 done!
GCA_001827545.1 done!
GCA_001827345.1 done!
GCA_001826735.1 done!
GCA_001826495.1 done!
GCA_003187085.1 done!
GCA_002525085.1 done!
GCA_003189325.1 done!
GCA_003607195.1 done!
GCA_003390165.1 done!
GCA_001826475.1 done!
GCA_900162205.1 done!
GCA_003812545.1 done!
GCA_003609395.1 done!
GCA_003608255.1 done!
GCA_003607595.1 done!
GCA_003607435.1 done!

GCA_002197325.1 done!
GCA_002135445.1 done!
GCA_001018245.1 done!
GCA_900459155.1 done!
GCA_900459015.1 done!
GCA_900458975.1 done!
GCA_900074465.1 done!
GCA_900073505.1 done!
GCA_006543205.1 done!
GCA_006543125.1 done!
GCA_004349485.1 done!
GCA_004120385.1 done!
GCA_002025025.1 done!
GCA_003859725.1 done!
GCA_003859585.1 done!
GCA_003859485.1 done!
GCA_003859445.1 done!
GCA_003812445.1 done!
GCA_003605605.1 done!
GCA_003174195.1 done!
GCA_002930685.1 done!
GCA_002278755.1 done!
GCA_900459245.1 done!
GCA_002135545.1 done!
GCA_003859675.1 done!
GCA_001018225.1 done!
GCA_000289435.1 done!
GCA_001017045.1 done!
GCA_000289695.1 done!
GCA_000289575.1 done!
GCA_000289295.1 done!
GCA_000288355.1 done!
GCA_001018055.1 done!
GCA_000288055.1 done!
GCA_000288035.1 done!
GCA_000287995.1 done!
GCA_000186445.1 done!
GCA_000167735.1 done!
GCA_900683545.1 done!
GCA_900458995.1 done!
GCA_003859735.1 done!
GCA_002882745.1 done!
GCA_002882125.1 done!
GCA_002881435.1 done!
GCA_002881395.1 done!
GCA_002881

GCA_000347895.1 done!
GCA_000339695.1 done!
GCA_000339675.1 done!
GCA_000339655.1 done!
GCA_000339635.1 done!
GCA_000339615.1 done!
GCA_000339575.1 done!
GCA_000339535.1 done!
GCA_000339495.1 done!
GCA_000339795.1 done!
GCA_000496535.1 done!
GCA_000522585.1 done!
GCA_000522565.1 done!
GCA_003466855.1 done!
GCA_002995555.1 done!
GCA_002213005.1 done!
GCA_002212845.1 done!
GCA_002179995.1 done!
GCA_002155285.1 done!
GCA_002083175.2 done!
GCA_001703615.1 done!
GCA_001625005.1 done!
GCA_001068415.1 done!
GCA_000496555.1 done!
GCA_000522905.1 done!
GCA_000522845.1 done!
GCA_000522825.1 done!
GCA_000522805.2 done!
GCA_000522785.1 done!
GCA_000522725.1 done!
GCA_000522705.1 done!
GCA_000522665.1 done!
GCA_000522645.1 done!
GCA_000522625.1 done!
GCA_000339475.1 done!
GCA_000522885.1 done!
GCA_000339435.1 done!
GCA_000271865.1 done!
GCA_000339375.1 done!
GCA_000229545.2 done!
GCA_000229485.2 done!
GCA_000229465.2 done!
GCA_000229445.2 done!
GCA_000229425.2 done!
GCA_000229405.2 done!
GCA_000229

GCA_900990425.1 done!
GCA_900989605.1 done!
GCA_900991495.1 done!
GCA_900989325.1 done!
GCA_900989135.1 done!
GCA_900989015.1 done!
GCA_900988945.1 done!
GCA_900988935.1 done!
GCA_900988595.1 done!
GCA_900988005.1 done!
GCA_900987065.1 done!
GCA_900991635.1 done!
GCA_900995535.1 done!
GCA_900991805.1 done!
GCA_900996055.1 done!
GCA_900995905.1 done!
GCA_900995785.1 done!
GCA_900995705.1 done!
GCA_900995635.1 done!
GCA_900995555.1 done!
GCA_900987015.1 done!
GCA_900995485.1 done!
GCA_900995475.1 done!
GCA_900995415.1 done!
GCA_900991765.1 done!
GCA_900994665.1 done!
GCA_900993605.1 done!
GCA_900993555.1 done!
GCA_900992665.1 done!
GCA_900992645.1 done!
GCA_900992525.1 done!
GCA_900992345.1 done!
GCA_900992305.1 done!
GCA_900992205.1 done!
GCA_900992055.1 done!
GCA_900991835.1 done!
GCA_900994235.1 done!
GCA_900986895.1 done!
GCA_900990455.1 done!
GCA_900986215.1 done!
GCA_900475875.1 done!
GCA_900475615.1 done!
GCA_900475155.1 done!
GCA_900474995.1 done!
GCA_900474965.1 done!
GCA_900474

GCA_000441115.1 done!
GCA_000441095.1 done!
GCA_000440675.1 done!
GCA_000440375.1 done!
GCA_000440195.1 done!
GCA_000440175.1 done!
GCA_000440135.1 done!
GCA_000440095.1 done!
GCA_000440075.1 done!
GCA_000231325.2 done!
GCA_001293335.1 done!
GCA_000941215.1 done!
GCA_000937615.1 done!
GCA_002760225.1 done!
GCA_000441315.1 done!
GCA_000441355.1 done!
GCA_000440355.1 done!
GCA_000441495.1 done!
GCA_000441475.1 done!
GCA_002760075.1 done!
GCA_002760055.1 done!
GCA_002759775.1 done!
GCA_002759735.1 done!
GCA_002759725.1 done!
GCA_002759695.1 done!
GCA_002759675.1 done!
GCA_002760135.1 done!
GCA_000947535.1 done!
GCA_000947235.1 done!
GCA_000945455.1 done!
GCA_000944335.1 done!
GCA_000943055.1 done!
GCA_000941495.1 done!
GCA_000937695.1 done!
GCA_002759625.1 done!
Streptococcus suis_P done
GCA_002963905.1 done!
GCA_002964225.1 done!
GCA_003144175.1 done!
GCA_003285245.1 done!
GCA_003352205.1 done!
GCA_003933915.1 done!
GCA_004922805.1 done!
GCA_006568995.1 done!
GCA_004922855.1 done!
GCA_00